# Ejercicio 1

Primero, generamos una población de vectores aleatorios.

Pasamos el parámetro $k$ (generamos una población inicial de $2k$ individuos), y el hipercubo al que pertenecen los vectores (o sea, la dimensión $n$ y el intervalo $[a,b]$ al que pertenecen cada una de las entradas de los vectores).

In [14]:
'''Función que nos genera una población inicial de 2k individuos y 
   recibe como parámetros: La 'k' y el hipercubo al que pertenecen 
   los vectores (o sea, la dimensión 'n' y el intervalo [a,b] al que
   pertenecen cada una de las entradas de los vectores).'''

import numpy as np

def Generar_Poblacion(k, n, intervalo):
    
   a, b = intervalo

   # Se genera una matriz de tamaño (2k, n) con valorios aleatorios
   # uniformes (o sea, una población inicial de 2k individuos donde 
   # cada uno vive en Rn):
   poblacionMatriz = np.random.uniform(a,b, size=(2*k, n))

   # Para facilitarnos la vida, convertimos a la matriz en una lista
   # de listas:
   poblacionLista = poblacionMatriz.tolist()

   return poblacionLista


Sea $v_i$ un individuo.

Definimos una función de aptitud dada por $g(v_i)=e^{-10^{-9}f(v_i)}$, donde $f$ es la función de prueba que estemos interesados en minimizar.

La definimos de este manera porque (**AQUÍ USA TUS PODERES DE REDACCIÓN**):
1. Queremos minimizar nuestras funciones (las $f$). 
2. La función exponencial, es monótona decreciente cuando se toma exponente negativo (o sea $e^{-y}$). Así, cuando maximizamos  $e^{-f(v_i)}$ estamos minimizando $f(v_i)$ de manera indirecta. Pues, valores "pequeños" de $f(v_i)$ nos dan valores "grandes" de $e^{-f(v_i)}$. Que es lo que queremos porque nos interesa que los individuos que den evaluaciones de $f$ pequeñas (sin importar si son positivas o negativas) tengan mejores aptitudes asociadas (o sea, valores de $e^{-f}$ grandes).
3. Multiplicamos $-f(v_i)$ por $10^{-9}$ porque si tenemos la mala suerte de que al evaluar $f$ obtenemos valores muy grandes, $e^{-f(v_i)}$ nos da valores muy pequeños que la computadora nos llega a considerar como $0$, lo que causa problemas porque entonces en `aptitud/sumaAptitudes` vamos a estar dividiendo $0$ entre $0$ y eso no se puede.

Más adelante, vamos a hacer la selección de padres de la siguiente generación por medio del método de la ruleta. (**AQUÍ ESTARÍA BIEN UNA BREVE EXPLICACIÓN DE QUÉ ES Y CÓMO FUNCIONA, CON QUE PONGAS ALGO DE CHATGPT**)

La `sumaAptitudes` nos da el tamaño de la ruleta (a partir de aquí, ruleta=pastel).

Se evalúan los individuos mediante la **función de aptitud** y se les asigna un "pedazo del pastel" en la ruleta (proporcional a "qué tan buenos" son):
* Tendremos una `sumaAptitudes` **($F$)** que nos va a determinar 'el tamaño del pastel completo'. Éste es la suma de todas las aptitudes ($e^{-10^{-9}f(v_i)}$).
    * Para cada individuo $v_i$, tendremos una **probabilidad de selección asociada** $p_i$. Ésta está dada por: $\frac{e^{-10^{-9}f(v_i)}}{F}$.
    * El "pedazo del pastel" que le corresponde a un individuo es su probabilidad de ser seleccionado para la siguiente generación. Ésta depende de su aptitud entre la suma de aptitudes. Vamos a considerar una probabilidad acumulativa para facilitar la implementación.
        * Por "probabilidad acumulativa" nos referimos a:
        
            Si tienes una lista de individuos con sus probabilidades $p_i$ las probabilidades acumulativas $q_i$ crean una escala o 'intervalos' en los que se ubica cada individuo.

            Por ejemplo si el **Individuo 1** tiene una probabilidad de 0.1, y el **Individuo 2** tiene una probabilidad de 0.2, entonces: 
            
            $q_1 = 0.1$ (el intervalo del **Individuo 1** es de 0 a 0.1) 
            
            $q_2 = 0.1 + 0.2 = 0.3$ (el intervalo del **Individuo 2** es de 0.1 a 0.3)

In [15]:
'''Definimos una función de aptitud que recibe como parámetros una
   función de prueba y una población de 2k individuos y retorna una
   lista con las probabilidades de selección (p_i) de todos los 
   individuos de la población. Además de que nos da al mejor individuo
   en la población.'''

def Probabilidades_De_Seleccion_Y_Mas_Apto(funcion, poblacion):
    # Generamos una lista con las evaluaciones en la función objetivo
    evaluaciones = [funcion(individuo) for individuo in poblacion]

    #print("Evaluaciones:", [float(eval) for eval in evaluaciones], "\n")

    # Generamos una lista de las aptitudes de los individuos en la 
    # población.
    aptitudes = [np.exp(1e-9*(-evaluacion)) for evaluacion in evaluaciones]

    #print("Aptitudes:", [float(apt) for apt in aptitudes], "\n")

    # Encontramos el índice del mejor individuo (menor evaluación)
    mejorIndice = np.argmax(aptitudes)
    masApto = poblacion[mejorIndice]

    #print("El individuo más apto fue:", masApto, "\n")

    # Obtenemos el 'tamaño del pastel':
    sumaAptitudes = sum(aptitudes)

    # Calculamos las probabilidades de selección (p_i) para todos los
    # individuos:
    probabilidades = [(aptitud/sumaAptitudes) for aptitud in aptitudes]

    #print("Probabilidades de selección:", [float(pi) for pi in probabilidades])


    return probabilidades, masApto


In [16]:
'''Definimos una función que recibe la lista de probabilidades 
   de selección y nos retorna la que corresponde a la de las 
   probabilidades acumulativas.'''

def Probas_Acumulativas(probabilidades):

    # Calculamos las probabilidades acumulativas (q_i) para todos los
    # individuos:
    probasAcumulativas = []
    sumaAcumulada = 0

    for proba in probabilidades:
        sumaAcumulada += proba
        probasAcumulativas.append(sumaAcumulada)

    #return probasAcumulativas
    return probasAcumulativas


Con probabilidad uniforme (pensando en que estamos lanzando un dardo a la ruleta) generamos un número $r \in [ 0,1 ]$. Si $r < q_i$ (si el dardo cae en el pedazo de pastel correspondiente al individuo $v_i$), el individuo $v_i$ es elegido como el primer padre.

La selección por método de la ruleta por sí sola causa convergencia prematura, pues si tenemos individuos muy buenos contra muy malos, los muy buenos tienen asignados un pedazo de la ruleta "grande". Así que cuando hacemos la elección de padres, llegan a haber ocasiones donde se nos generan demasiados clones (recordemos que una pareja de padres nos da dos hijos que van a reemplazarlos).

Si tenemos una población de tamaño $2k$, para reemplazarla en la siguiente generación elegimos $k$ parejas. Por sí solo el método de selección por ruleta permite que las $k$ parejas estén formadas por el mismo padre (algo así como: `(individuoMasApto, individuoMasApto)` en las $k$ parejas). Esto provoca que en pocas iteraciones el algoritmo deje de mejorar (pues sólo nos estará generando clones salvo -tal vez- pequeñas mutaciones).

Para tratar de contrarrestar esto, le metemos como parámetro un `maxIntentos=100` para seleccionar un segundo padre distinto al primero; lo fijamos con este número por dos razones:
1. Nuestras poblaciones, en principio, están pensadas para ser de 100 a 200 individuos (no tiene mucho sentido hacer más intentos de selección de padre que tamaño de la población).
2. Aún cuando le damos números más grandes (digamos `maxIntentos=1000`), si hay ciertos individuos con probabilidades de selección muy buenas contra otros de probabilidades muy malas, se nos seguirán generando clones porque la probabilidad de obtener padres distintos ya está fijada y es muy pequeña. Así que, asignar un número muy grande a `maxIntentos` sólo aumenta innecesariamente la complejidad del algoritmo y no nos da mejores resultados.

Usamos `tol=1e-9` y `math.isclose()` porque las comparaciones de números de punto flotante (números decimales) pueden ser imprecisas, debido a cómo las computadoras los manejan. 

`math.isclose()` permite manejar esta imprecisión al verificar si `r` y `probAcumulada` son prácticamente iguales. Esto evita errores en situaciones donde `r` cae "justo en el filo" que separa las probabilidades acumulativas entre dos individuos.

`tol=1e-9`  define la tolerancia. Si `r` está a una distancia menor que $10^{-9}$ de `probAcumulada`, lo tratamos como si fueran iguales.

En general, en este tipo de algoritmos, cuando $r$ cae exactamente en el límite entre dos intervalos, lo más común es que se elija el individuo cuyo intervalo incluye el valor en el extremo inferior. 

Esto significa que si $r$ es exactamente igual a la probabilidad acumulativa de un individuo, se selecciona el individuo más pequeño, es decir, el que corresponde a ese valor acumulativo. O sea, los intervalos son inclusivos en su límite inferior y exclusivos en su límite superior. Por ejemplo, un intervalo $[0.2, 0.5)$ incluiría 
$r=0.2$, pero no $r=0.5$.

In [17]:
'''Estas funciones son las que nos ayudan a generar a los padres mediante
   el uso de la ruleta.'''
   

'''La primera es para seleccionar padres (uno) y recibe como parámetro 
   la lista de las probabilidades acumulativas.
   
   Nos retornará el índice en la lista que le corresponde al padre 
   elegido con esa regla.'''

import random
import math

def Seleccionar_Padre_Ruleta(probasAcumuladas):
   r = random.random() # Generamos un número aleatorio entre 0 y 1

   tol=1e-9
   for i, probAcumulada in enumerate(probasAcumuladas):
      if r<= probAcumulada or math.isclose(r, probAcumulada, abs_tol=tol):
         return i # Índice del individuo seleccionado.


'''Le segunda es para generar la lista de padres elegidos para 
   reproducirse. Necesitamos 2k nuevos individuos, así que se van a 
   elegir k parejas (cada par de padres produce dos hijos).
   La función recibe como parámetros: la lista de individuos, las
   probabilidades aculativas y un máximo de intentos para intentar evitar
   la generación de clones (en este caso, vamos a considerar un máximo
   de 100 para cada elección de un segundo padre, pero tenemos la 
   apertura de modificarlo después).'''

def Generar_Parejas(individuos, probasAcumuladas, maxIntentos=100):
    parejas = []
    clonesGenerados = 0  # Contador de clones generados
    numIndividuos = len(individuos)

    if numIndividuos < 2:
        print("Error: No hay suficientes individuos para generar parejas.")
        return parejas, clonesGenerados

    # Como len(individuos)=2k, le pedimos que elija a len(individuos)/2
    # parejas de padres (que nos generarán 2k hijos).
    parejasNecesarias = int(numIndividuos/2)
    #print(f"Se necesitan {parejasNecesarias} parejas de padres.")

    for _ in range(parejasNecesarias):
        padre1_idx = Seleccionar_Padre_Ruleta(probasAcumuladas)

        intentos = 0
        while True:
            padre2_idx = Seleccionar_Padre_Ruleta(probasAcumuladas)

            if padre1_idx != padre2_idx:
                break  # Padres diferentes, salir del ciclo
            
            intentos += 1
            if intentos >= maxIntentos:  # Después de varios intentos, permitir clones
                padre2_idx = padre1_idx
                clonesGenerados += 2 # Cada pareja produce dos hijos
                #print(f"Se ha generado un clon tras {intentos} intentos.")
                break  # Salir del ciclo permitiendo el clon

        # Agregamos la pareja solo si hay padres válidos
        if padre1_idx is not None and padre2_idx is not None:
            parejas.append((individuos[padre1_idx], individuos[padre2_idx]))
        else:
            print("Error: No se pudieron seleccionar padres válidos.")

    return parejas, clonesGenerados


En principio retornamos el número de `clonesGenerados`, esto sólo nos sirvió para hacer pruebas empíricas y así determinar un valor para `maxIntentos`.

Las siguientes son las funciones auxiliares para la codificación en binario de las parejas de padres.

In [18]:
'''Funciones auxiliares para la codificación en binario.'''

def codifica_real(x, n_bit, intervalo):
    """
    Codifica un número real en el intervalo [a,b] utilizando nBit bits y una partición uniforme en [a, b].

    Parámetros:
    x: número a real a codificar.
    nBit: número de bits a utilizar.
    a: Extremo izquierdo del intervalo.
    b: Extremo derecho del intervalo.

    Retorno:
    Arreglo binario que representa al número real x con nBit bits en el intervalo [a, b].
    """
    
    a, b = intervalo
    
    # Calcula la precisión de la representación.
    precision = (b - a) / ((2 ** n_bit) - 1)

    # Asegura que el número esté dentro del rango de la partición.
    x = max(a, min(b, x))

    # Calcula el índice del número en la partición.
    index = int((x - a) / precision)

    # Codifica el índice a binario usando nuestro codigo para codificar naturales.
    if index < 0 or index >= (1 << n_bit):
        raise ValueError(f"Índice fuera del rango representable con {n_bit} bits.")

    x_binario = [0] * n_bit
    for i in range(n_bit - 1, -1, -1):
        x_binario[i] = index & 1
        index >>= 1

    # Devuelve el arreglo binario que representa al número real 'x'.
    return x_binario

def codifica_vector(vector_reales, n_bit, intervalo):
    """
    Codifica un vector de números reales en un vector de vectores binarios utilizando nBit bits.

    Parámetros:
    vector_reales: Arreglo de números reales a codificar
    dim_x: Diimensión del vector vector_reales
    nBit: Número de bits a utilizar para las entradas de nuestro arreglo.
    a: Extremo izquierdo del intervalo.
    b: Extremo derecho del intervalo.

    Retorno:
    Arreglo de arreglos arreglos binarios, donde cada subarreglo representa un número real codificado en binario
    """
    
    dim_x = len(vector_reales)

    # Inicializa un arreglo vacío para almacenar los vectores binarios.
    vector_binario = []

    # Itera sobre cada entrada de nuestro vector de reales.
    for i in range(dim_x):
        numero = vector_reales[i]

        # Codifica cada número utilizando la función codifica
        binario = codifica_real(numero, n_bit, intervalo)

        # Añade el resultado codificado al vector principal
        vector_binario.append(binario)

    # Devuelve un arreglo de arreglos binarios.
    return vector_binario


#print(f"El vector {[1.4,1.2,3,1.5674]} en binario con {10} bits es:\n")
#print(codifica_vector([1.4,1.2,3,1.5674],10,(1,5)))


Con ellas codificamos a los padres como vectores de bits (cada uno) del estilo: `[[0,1,1, ...], ..., [1, 0, 1, ...]]`.

La función recibe como parámetros la lista de las tuplas de las parejas de padres, el número de bits deseados para la codificación en binario y el intervalo donde están cada una de las entradas de los vectores.

In [19]:
'''Codificación de los padres en vectores binarios del estilo
   [[0,1,1,1,...], ..., [1, 1, 1, 0,...]]'''

def Padres_Binarios(parejas, nBits, intervalo):
   parejasBinarias = []

   for padre1, padre2 in parejas:

      padre1Bin = codifica_vector(padre1, nBits, intervalo)
      padre2Bin = codifica_vector(padre2, nBits, intervalo)

      parejasBinarias.append((padre1Bin, padre2Bin))
   
   return parejasBinarias


Vamos ahora a considerar los operadores de generación de crías y mutación, estos los vamos a aplicar cromosoma a cromosoma (O sea, los puntos de cruza se van a considerar dentro de cada uno de los vectores de 0s y 1s).

O sea, si nuestra pareja de padres es: `(padre1, padre2) = ([[1, 0], [0, 1]], [[1, 1], [0, 0]])`, el primer cromosoma del `padre1` es `[1, 0]`

En la función de cruza de $N$ puntos:

1. Metemos como parámetro el número de cortes que queremos hacerle a los padres en su codificación binaria para la cruza (personalmente, así interpreté la instrucción del ejercicio).
2. En caso de que `nCortes` sea demasido grande (tengamos más cortes deseados que el número de `nBits` de la codificación binaria), se hará el mayor número de cortes posible, o sea `nBits - 1` (O sea, estamos asumiendo que el número de cortes deseado es el mayor posible y sólo hubo un descuido). De aquí la línea `n = min(nCortes, (nBits - 1))`. 
3. La `n` es la misma para todas las parejas de padres (puesto que es un parámetro de la función, si pudiera variar no tendría sentido pedirla como parámetro).
4. Para cada pareja de padres generamos los `n` cortes de manera aleatoria (aunque por cuestiones de uniformidad todos los padres tienen el mismo número de cortes por cromosoma, estos no tienen que estar en el mismo sitio para todas las parejas de padres). De aquí la línea `puntosCruce = sorted(random.sample(range(1, len(cromo1)), n))` dentro del ciclo para las parejas de padres.

Como ya sabemos que nuestro algoritmo (muy probablemente) va a presentar convergencia prematura, de manera aleatoria (sin obtener su evaluación previamente), eliminamos a uno de los $2k$ hijos generados. Esta medida tiene la finalidad de permitir que (posiblente) convervemos individuos con valores "malos" en la función objetivo y tengamos más diversidad en la población.

In [20]:
'''Función que realiza la cruza de n puntos. Recibe como parámetros: la
   lista de los padres (en arreglos binarios), el número de cortes que 
   se quieren hacer por cromosoma y el número de bits usados en la 
   codificación binaria. Retorna una lista de 2k hijos (como arreglos 
   binarios).'''

def Cruzar_N_Puntos(padresBinarios, nCortes, nBits):
    
    hijos = [] # Lista donde se guardará a los 2k hijos
    
    # Los `nCortes` son un parámetro recibido e idealmente es el que se
    # usará, en caso de que `nCortes` sea demasido grande, se hará el 
    # mayor número de cortes posible, o sea `nBits - 1`
    n = min(nCortes, (nBits - 1))
    
    for padre1, padre2 in padresBinarios:
        
        # Los hijos se van a crear cruzando cada cromosoma de los padres. 
        # Recordemos que cada pareja produce dos hijos.
        hijo1 = []
        hijo2 = []
        
        # Cruzamos cada cromosoma individualmente
        for cromo1, cromo2 in zip(padre1, padre2): # `zip(padre1, padre2)` toma los cromosomas 
                                                   # correspondientes de ambos padres al mismo 
                                                   # tiempo, es decir, empareja los cromosomas 
                                                   # que están en las mismas posiciones de las 
                                                   # listas `padre1` y `padre2`.
            
            # Determinar los puntos de cruce
            puntosCruce = sorted(random.sample(range(1, len(cromo1)), n))

            #print(puntosCruce) # En la primera iteración, si tenemos [1, 5]
                               # esto quiere decir que en el primer cromosoma
                               # de la pareja de padres, se van a hacer cruces
                               # en tres pedazos determinados por los dos cortes
                               # en las posiciones 1 y 5.
            
            # Inicializamos cromosomas vacíos para los hijos
            hijo1Cromo = []
            hijo2Cromo = []
            
            ultimoPunto = 0 # Punto en el que vamos para los cruces, inicia en 0.
            
            switch = False  # Esta bandera indica si debemos cambiar de padre
            
            # Alternamos entre segmentos de los padres en los puntos de cruza
            for punto in (puntosCruce + [len(cromo1)]): # Añadimos el valor len(cromo1) 
                                                        # al final de la lista de puntos de 
                                                        # cruza para asegurarnos de que el 
                                                        # último segmento después del último 
                                                        # punto de corte también se incluya. 
                                                        # De esta manera, recorremos todo el 
                                                        # cromosoma.
                
                # Seleccionamos los segmentos entre el último punto y el punto actual
                if switch:
                    hijo1Cromo += cromo2[ultimoPunto:punto] # El cromosoma del hijo1 recibe 
                                                            # parte del cromosoma del segundo
                                                            # padre.
                    hijo2Cromo += cromo1[ultimoPunto:punto] # El cromosoma del hijo2 recibe 
                                                            # parte del cromosoma del primer
                                                            # padre.
                else:
                    hijo1Cromo += cromo1[ultimoPunto:punto] # El cromosoma del hijo1 recibe 
                                                            # parte del cromosoma del primer
                                                            # padre.
                    hijo2Cromo += cromo2[ultimoPunto:punto] # El cromosoma del hijo2 recibe 
                                                            # parte del cromosoma del segundo
                                                            # padre.
                
                # Cambiamos el segmento
                switch = not switch # De esta manera garantizamos que vamos a 
                                    # ir alternando.
                ultimoPunto = punto # Nos movemos al siguiente punto de corte.
            
            # Añadimos el nuevo cromosoma a los hijos
            hijo1.append(hijo1Cromo)
            hijo2.append(hijo2Cromo)
        
        # Añadimos los dos hijos generados a la lista de hijos
        hijos.append(hijo1)
        hijos.append(hijo2)
    
    # Para este punto ya tenemos la lista de los hijos, así que lo que toca es
    # eliminar uno para hacerle espacio al individuo más apto.
    index_a_eliminar = random.randint(0, len(hijos)-1) # De manera aleatoria, 
                                                       # elegimos un índice para
                                                       # quitar un hijo de la lista.
    #print(hijos)
    #print(f'Eliminando al hijo en la posición: {index_a_eliminar}')
    del hijos[index_a_eliminar] # Borramos al hijo elegido.
    
    return hijos # Retornamos una lista de 2k-1 hijos (que era lo que queríamos)


Ahora, en la lista de $2k-1$ hijos, vamos a implementar un operador de mutación 1-flip.
Lo haremos de la siguiente forma: 
1. Para cada hijo elegimos un cromosoma al azar.
2. Dentro del cromosoma elegido, elegimos un bit al azar.
3. Tenemos una cierta probabilidad de mutación $p_m$, generamos un número aleatorio $r$ entre 0 y 1. Si $r<p_m$, entonces el bit elegido cambia su valor ($0 \to 1$ y $1 \to 0$).
4. Generamos una nueva lista de hijos (posiblemente) mutados.

In [21]:
'''Función que realiza la mutación de un bit elegido al azar: Por cada hijo
   elegimos un cromosoma al azar, dentro de ese cromosoma elegimos un bit al
   azar, generamos un número aleatorio r en [0,1]. Si r<probaMutar, cambiamos 
   el valor del bit, en otro caso el bit se queda igual.
   Recibimos como parámetros: la lista de los hijos (en binario) y una 
   probabilidad de mutación. La función retorna la lista (en binario) de los hijos
   que (posiblemente) han sido mutados.'''
   

import copy

def Mutador_1_flip(hijos, probaMutar):
    
    hijosMutados = []
    #huboMutacion = False # Variable para verificar si hubo al menos una mutación.

    for hijo in hijos:
        hijo = copy.deepcopy(hijo)  # Hacemos una copia profunda del hijo para no modificar el 
                                    # original. si no hacemos una copia de los hijos antes de 
                                    # realizar mutaciones, estaremos modificando las listas 
                                    # originales directamente. Lo que puede causarnos problemas
                                    # si luego intentamos usar los hijos originales en alguna 
                                    # otra operación.          

        if isinstance(hijo[0], list):  # Si el primer elemento es una lista, tenemos listas de 
                                       # cromosomas (o sea, la dimensión del problema es al 
                                       # menos 2).
            # Elegimos un cromosoma al azar dentro del hijo

            cromElegido_index = random.randint(0, len(hijo)-1) # Elegimos un índice al azar en 
                                                           # `hijo`. Este corresponde a un 
                                                           # cromosoma.
            cromElegido = hijo[cromElegido_index] # Fijamos el cromosoma elegido.

            bit_index = random.randint(0, len(cromElegido)-1) # Elegimos un índice al azar en 
                                                              # el cromosoma elegido, este 
                                                              # corresponde a alguno de los 
                                                              # nBits.
        
            r = random.random() # Generamos un número aleatorio entre 0 y 1.

            # Si el número aleatorio es menor que la probabilidad de mutación p_m
            if r < probaMutar:
            
                # Hacemos flip al bit: si es 0 lo cambiamos a 1, si es 1 lo cambiamos a 0
                cromElegido[bit_index] = 1 if cromElegido[bit_index] == 0 else 0

                #huboMutacion = True

            # Añadimos el cromosoma (posiblemente mutado) a la lista de hijosMutados
            hijosMutados.append(hijo)

        else: # Si el  no es una lista de listas, es directamente un cromosoma (o sea, la
              # dimensión del problema es 1).
        
            bit_index = random.randint(0, len(hijo)-1) # Elegimos un bit al azar dentro del 
                                                       # cromosoma (en este caso, cada hijo
                                                       # es su propio cromosoma).
            
            r = random.random() # Generamos un número aleatorio entre 0 y 1.

            # Si el número aleatorio es menor que la probabilidad de mutación p_m
            if r < probaMutar:
                hijo[bit_index] = 1 if hijo[bit_index] == 0 else 0 # Flip del bit.

                #huboMutacion = True

            # Añadimos el cromosoma (posiblemente mutado) a la lista de hijosMutados
            hijosMutados.append(hijo)
    
    #print("Hubo mutación:", huboMutacion)

    return hijosMutados


Estas son las funciones auxiliares que nos servirán para pasar de nuestros arreglos de bits a vectores de números reales.

In [22]:
'''Funciones auxiliares para pasar a los hijos (posiblemente) mutados de su 
   codificación en arreglos de bits a vectores de números reales.'''

def decodifica(x_cod, nBits, intervalo):
    """
    Decodifica un vector de bits como un número real en el intervalo [a, b].

    Parámetros:
    x_cod: vector de bits a decodificar
    nBits: número de bits a utilizar
    intervalo: De la forma [a, b]

    Retorno:
    número real decodificado
    """

    a, b = intervalo
    precision = (b - a) / ((2 ** nBits) - 1)  # Calcula la precisión de la representación
    indice = int(''.join(map(str, x_cod)), 2)  # Convierte el vector de bits en un número entero
    x_dec = a + indice * precision  # Calcula el número real decodificado

    return x_dec

def decodifica_vector(vector_binario, nBits, intervalo):
    """
    Decodifica un vector de arreglos binarios en un vector de números reales.

    Parámetros:
    vector_binario: lista de listas de enteros, donde cada sublista representa un número codificado en binario
    nBits: número de bits a utilizar para cada número
    a: límite inferior de la partición
    b: límite superior de la partición

    Retorno:

    lista de números reales decodificados
    """

    dim_x = len(vector_binario)


    vector_reales = []

    for i in range(dim_x):
        binario = vector_binario[i]
        # Decodifica cada número utilizando la función decodifica
        real = decodifica(binario, nBits, intervalo)
        # Añade el resultado decodificado al vector principal
        vector_reales.append(real)

    return vector_reales


Esta función en la que realiza la decodificación de los $2k - 1$ hijos (posiblemente ya mutados).

In [23]:
'''Función que decodifica a los hijos mutados para el reemplazo en la 
   siguiente generación. Recibe como parámetros a loss hijos (posiblemente) 
   mutados (codificados en bits), la cantidad de nBits usada en la codificación
   y el intervalo de nuestra función objetivo.   
   Retorna la lista de los hijos como vectores de números reales.'''

def Hijos_Decodificados(hijosMutados, nBits, intervalo):
    hijosDecodificados = []
    for hijoMutado in hijosMutados:
        hijoDecodificado = decodifica_vector(hijoMutado, nBits, intervalo)
        hijosDecodificados.append(hijoDecodificado)

    return(hijosDecodificados)


La siguiente función es la que nos da la siguiente generacion completa. Recibe como parámetros la lista de los $2k-1$ hijos decodificados como vectores de números reales y agrega a la lista al individuo más apto (que habíamos guardado previamente en la variable `masApto`) para tener el reemplazo con elitismo.

In [24]:
'''Función que nos da la siguiente generacion completa. Recibe como
   parámetros la lista de los 2k-1 hijos decodificados como vectores de 
   números reales y agrega a la lista al individuo más apto para 
   tener el reemplazo con elitismo.'''

def Siguiente_Gen(hijosDecodificados, masApto):
    siguenteGeneracion = hijosDecodificados + [masApto]
    return siguenteGeneracion


La función `Algoritmo_Genetico` es la encargada de realizar una ejecución del algoritmo genético en sí, Recibe como parámetros: 
* la función objetivo a minimizar,
* el intervalo de ésta, 
* la dimensión del problema, 
* la mitad del tamaño de la población (pues vamos a tomar $2(kindividuos)$), 
* la cantidad de bits para la codificación en arreglos binarios, 
* el número de cortes para el operador de reproducción de cruza, 
* la probabilidad de mutación de un bit y 
* el número de iteraciones a realizar dentro de la ejecución (el número de generaciones).

In [25]:
'''Función que realiza una ejecución del algoritmo genético. Recibe como
   parámetros: la función objetivo a minimizar, el intervalo de ésta, la
   dimensión del problema, la mitad del tamaño de la población (pues vamos
   a tomar 2*(kindividuos)), la cantidad de bits para la codificación en 
   arreglos binarios, el número de cortes para el operador de reproducción 
   de cruza, la probabilidad de mutación de un bit y el número de iteraciones
   a realizar dentro de la ejecución (el número de generaciones).'''

def Algoritmo_Genetico(funcion, intervalo, dimension,   # Cuando tienes una función que
                       kindividuos, nBits, nCortes,     # tiene muchos parámetros, la
                       probaMutar, iteraciones):        # puedes poner así para que sea
                                                        # más legible.
    
    contadorIteraciones = 0 # Contamos las iteraciones.

    # Generamos a nuestra población inicial:
    poblacion = Generar_Poblacion(kindividuos, dimension, intervalo) 
    
    while contadorIteraciones <= iteraciones:

        # Calculamos las probabilidades de selección y obtenemos el individuo más apto
        probas_pi, masApto = Probabilidades_De_Seleccion_Y_Mas_Apto(funcion, poblacion)

        # Calculamos las probabilidades acumulativas
        probasAcumulativas_qi = Probas_Acumulativas(probas_pi)

        # Seleccionamos a los padres (ponemos `_` porque no nos interesa el número de 
        # clones).
        padres, _ = Generar_Parejas(poblacion, probasAcumulativas_qi)

        # Convertimos a los padres en arreglos de bits.
        padresBinarios = Padres_Binarios(padres, nBits, intervalo)

        # Cruzamos a los padres para generar hijos (como arreglos binarios).
        hijosBinarios = Cruzar_N_Puntos(padresBinarios, nCortes, nBits)

        # Aplicamos la mutación 1-flip a los hijos.
        hijosMutados = Mutador_1_flip(hijosBinarios, probaMutar)

        # Decodificamos a los hijos a su representación original como vectores de reales.
        hijosDecodificados = Hijos_Decodificados(hijosMutados, nBits, intervalo)

        # Generamos la siguiente generación, incluyendo el individuo más apto.
        poblacion = Siguiente_Gen(hijosDecodificados, masApto)

        contadorIteraciones += 1 # Incrementamos en uno el contador de las iteraciones.

    return masApto # Devolvemos la mejor solución encontrada.


**Funciones de prueba:**

In [26]:
def Sphere(lista):
  """
  Esta función recibe como parámetro un vector de n entradas, ingresado como una lista y devuelve el escalar que resulta de evaluar la función de la esfera en el vector.
  Se usa en el intervalo de busqueda [-5.12, 5.12] y su óptimo global está en f([0,..,0])=0

  Ejemplo de uso:
    > Sphere([0,0])

    > 0
    """
  suma = 0
  for x in lista:
    suma += x**2
  return suma

intervaloSphere = (-5.12, 5.12)


def Ackley(lista):
  """
  Esta función recibe como parámetro un vector de n entradas, ingresado como una lista y devuelve el escalar que resulta de evaluar la función Ackley en el vector.
  Se usa en el intervalo de busqueda [-30, 30] y su óptimo global está en f([0,..,0])=0

  Ejemplo de uso:
    > Ackley([0,0])

    > 0
    """
  suma1 = 0
  suma2 = 0
  n = len(lista)
  for x in lista:
    suma1 += (x**2)
    suma2 += np.cos( 2 * np.pi * x )
  return 20 + np.exp(1) - 20 * np.exp(-0.2 * np.sqrt((1/n)*suma1)) - np.exp((1/n)* suma2)

intervaloAckley = (-30, 30)


def Griewank(x):
  """
  Esta función recibe como parámetro un vector de n entradas, ingresado como una lista y devuelve el escalar que resulta de evaluar la función Griewank en el vector.
  Se usa en el intervalo de busqueda [-600, 600] y su óptimo global está en f([0,..,0])=0

  Ejemplo de uso:
    > Griewank([0,0])

    > 0
  """
  suma = 0
  m = 1 # se guardará la multiplicacion de cosenos y por eso se inicializa en 1
  n = len(x)
  for i in range(1, n+1 ):
    x_i = float(x[i-1])
    suma += ((x_i)**2) / 4000
    m = m * np.cos(x_i / np.sqrt(i))
  return 1 + suma - m

intervaloGriewank = (-600, 600)


def Rastrigin(lista):
  """
  Esta función recibe como parámetro un vector de n entradas, ingresado como una lista y devuelve el escalar que resulta de evaluar la función de Rastrigin en el vector.
  Tiene mínimo global en f([0,..,0])

  Ejemplo de uso:
    > Rastrigin([2,3,6,7])

    > 98.0

  """
  n = len(lista)
  suma = 0
  for x in lista:
    suma += x**2 - 10 * np.cos(2* np.pi *x)
  return 10 * n + suma

intervaloRastrigin = (-5.12, 5.12)


def Rosenbrock(lista):
  """
  Esta función recibe como parámetro un vector de n entradas, ingresado como una lista y devuelve el escalar que resulta de evaluar la función de Rosenbrock en el vector.
  Tiene un mínimo global en f(1, ..., 1)

  Ejemplo de uso:
    > Rosenbrock([1,1])

    > 0

  """
  n = len(lista)
  suma = 0
  for i in range(n-1):
    suma += 100*(lista[i+1] - (lista[i])**2)**2 + (lista[i]-1)**2
  return suma

intervaloRosenbrock = (-2.048, 2.048)


Veamos un ejemplo de ejecución de nuestro algoritmo para nuestras funciones de prueba (**ESTO YA NI AL CASO EN EL REPORTE**)

In [27]:
np.random.seed(1)

funciones = [(Sphere, "Sphere"), (Ackley, "Ackley"), (Griewank, "Griewank"),
             (Rastrigin, "Rastrigin"), (Rosenbrock, "Rosenbrock")]

intervalos = {"Sphere": intervaloSphere, "Ackley" : intervaloAckley,
              "Griewank": intervaloGriewank, "Rastrigin": intervaloRastrigin, 
              "Rosenbrock": intervaloRosenbrock}

dimension = 10
kindividuos = 70 # Tendremos una población del doble
nBits = 15
nCortes = 14
probaMutar = 0.45
iteraciones = 1000

for funcion, nombreFuncion in funciones:
    intervalo = intervalos[nombreFuncion]
    masApto = Algoritmo_Genetico(funcion, intervalo, dimension,   
                                 kindividuos, nBits, nCortes,     
                                 probaMutar, iteraciones)
    print(f"{nombreFuncion:<10}, {str(masApto):<10}, {funcion(masApto)}")


Sphere    , [-0.0023438215277566954, -0.0010937833796198504, -0.007343974120304075, 0.000781273842584973, 0.0001562547685169946, 0.0001562547685169946, -0.019844355601672525, 0.0001562547685169946, -0.005156407361064375, 0.035469832453382644], 0.001739803453180272
Ackley    , [0.008239997558519008, 0.23529770805993877, -0.0357066560869157, -0.013733329264198346, 0.03204443494979614, 0.019226660969877685, 0.03021332438123636, 0.004577776421399449, -0.11810663167210933, 0.03021332438123636], 0.6642705351283302
Griewank  , [0.531022064882336, 7.415997802667334, -5.621509445478637, 0.054933317056793385, -0.5676442762535316, -0.3112887966551625, 2.3621326334422292, -1.1902218695638567, -5.1087984862818985, 0.6042664876247272], 0.9992944758485716
Rastrigin , [-0.009844050416577765, -0.00859401226844092, -0.95987304300058, 1.918964812158574, 0.005156407361064375, 0.015469222083193124, 0.9583104953154082, 0.005156407361064375, 0.959560533463546, 0.005468916898098364], 8.787479125209515
Rosenbr

In [28]:
np.random.seed(1)

funcion = Rosenbrock

intervalo = intervaloRosenbrock

dimension = 10
kindividuos = 25 # Tendremos una población del doble
nBits = 15
nCortes = 2
probaMutar = 0.01
iteraciones = 100

masApto = Algoritmo_Genetico(funcion, intervalo, dimension,   
                                 kindividuos, nBits, nCortes,     
                                 probaMutar, iteraciones)
print(f"Rosenbrock:, {str(masApto):<10}, {funcion(masApto)}")


Rosenbrock:, [1.116096560563982, 1.0387191991943117, 0.8672139652699364, 0.40507486190374475, -0.02206317331461527, -0.49345255897701956, 0.9499664906765957, 1.1197216711935791, 1.1990990936002688, 1.5453596606341748], 108.47273872109665


In [29]:
import random

# Definir variables globales o necesarias
funciones = [(Sphere, "Sphere"), (Ackley, "Ackley"), (Griewank, "Griewank"),
             (Rastrigin, "Rastrigin"), (Rosenbrock, "Rosenbrock")]

intervalos = {"Sphere": intervaloSphere, "Ackley" : intervaloAckley,
              "Griewank": intervaloGriewank, "Rastrigin": intervaloRastrigin, 
              "Rosenbrock": intervaloRosenbrock}

dimension = 10
kindividuos = 70
nBits = 15
nCortes = 14
probaMutar = 0.45
iteraciones = 1000

ejecuciones = 5
j = 0



while j <= ejecuciones:
    # Agregamos un bucle que te genera una semilla diferente en cada ejecución
    for funcion, nombreFuncion in funciones:
        # Genera una semilla aleatoria
        semilla = random.randint(0, 10000)  # Puedes ajustar el rango si lo deseas
    
        # Establecer la semilla
        random.seed(semilla)
    
        # Si estás usando numpy para números aleatorios, asegúrate de hacer lo mismo
        # np.random.seed(semilla)
    
        intervalo = intervalos[nombreFuncion]
        masApto = Algoritmo_Genetico(funcion, intervalo, dimension,   
                                 kindividuos, nBits, nCortes,     
                                 probaMutar, iteraciones)
    
        # Imprimir la semilla y el resultado de la función
        print(f"Función: {nombreFuncion}, Mejor individuo: {str(masApto):<10}, Resultado: {funcion(masApto)}, Semilla: {semilla}")

        j += 1


Función: Sphere, Mejor individuo: [0.0017188024536878288, 0.0001562547685169946, -0.004843897824030385, -0.009531540879543776, 0.0001562547685169946, 0.0014062929166538396, 0.0001562547685169946, 0.0004687643055509838, -0.0007812738425858612, 0.012656636249885445], Resultado: 0.00028033937592152926, Semilla: 9314
Función: Ackley, Mejor individuo: [-0.06866664632099528, -0.026551103244116803, -0.0009155552842798897, 0.13275551622058757, 0.026551103244116803, 0.14923551133762558, -0.03753776665547548, -0.2572710348826561, 0.0357066560869157, -0.059511093478192834], Resultado: 0.9285610172401202, Semilla: 2209
Función: Griewank, Mejor individuo: [-0.018311105685597795, -4.083376567888422, -0.12817773979918456, 12.21350749229407, -14.117862483596355, 0.3479110080263581, -0.16479995117038015, -8.587908566545593, 8.001953184606464, -9.869685964537439], Resultado: 0.32024501655775583, Semilla: 9951
Función: Rastrigin, Mejor individuo: [-0.0010937833796198504, -0.01609424115726199, 0.985811334

In [30]:
np.random.seed(1)

funcion = Rosenbrock

intervalo = intervaloRosenbrock

dimension = 10
kindividuos = 25 # Tendremos una población del doble
nBits = 15
nCortes = 2
probaMutar = 0.01
iteraciones = 100

ejecuciones = 100
j = 0

while j <= ejecuciones:
    masApto = Algoritmo_Genetico(funcion, intervalo, dimension,   
                                 kindividuos, nBits, nCortes,     
                                 probaMutar, iteraciones)
    print(f"Rosenbrock:, {str(masApto):<10}, {funcion(masApto)}")
    j += 1


Rosenbrock:, [-0.4620766014587847, 0.3508232062746055, 0.49707766960661637, 0.5180783104953157, -0.011812860499893052, 0.431450666829432, -0.02193816949980132, -0.1804430066835534, 0.21981920834986424, -0.0840650654622026], 69.9722275636764
Rosenbrock:, [-0.7477103183080538, 0.5432040772728661, 0.44995123142185767, 0.5530793786431474, 0.15719229712820848, -0.298321604052858, 0.32144730979338965, -0.7192094485305336, 0.6878334910122992, 0.6392070070497757], 115.76715277802005
Rosenbrock:, [-0.4498262276070435, 0.5265785699026462, -0.026313303018280276, -0.05956431775872062, 0.322572344126713, -0.05193908505508582, 0.14019177831354712, -0.03268849757377845, 0.05456416516617324, -0.32819751579332856], 54.88957107982762
Rosenbrock:, [0.983467513046663, 0.7990868861964784, -0.20006860560930195, -0.04193877986999084, 0.3643236182744838, 0.17181774346140966, 0.25657032990508766, 0.09994054994354107, -0.459951536606952, 0.5483292336802275], 132.97097624296387
Rosenbrock:, [0.8249626758629112, 